In [1]:
import imp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
import seaborn as sns; sns.set()
from copy import deepcopy
import catboost as cgb
from bayes_opt import BayesianOptimization
from sklearn.datasets import load_boston
from skopt.space import Real, Categorical, Integer
from sklearn.metrics import r2_score , mean_squared_error
import lightgbm
url="C:/Users/louis/Downloads/Projet/"

In [2]:
# List of bank holidays
bank_holidays = [
    'January 1, 2011',
    'January 17, 2011',
    'February 21, 2011',
    'May 30, 2011',
    'July 4, 2011',
    'September 5, 2011',
    'October 10, 2011',
    'November 11, 2011',
    'November 24, 2011',
    'December 25, 2011',
    'December 26, 2011',
    'January 1, 2012',
    'January 2, 2012',
    'January 16, 2012',
    'February 20, 2012',
    'May 28, 2012',
    'July 4, 2012',
    'September 3, 2012',
    'October 8, 2012',
    'November 11, 2012',
    'November 12, 2012',
    'November 22, 2012',
    'December 25, 2012',
    'January 1, 2013',
    'January 21, 2013',
    'February 18, 2013',
    'May 27, 2013',
    'July 4, 2013',
    'September 2, 2013',
    'October 14, 2013',
    'November 11, 2013',
    'November 28, 2013',
    'December 25, 2013'
]
bank_holidays = pd.to_datetime(bank_holidays)

new_date_data = pd.DataFrame(index=pd.date_range(pd.to_datetime('2011-09-01'), pd.to_datetime('2013-05-03')))
new_date_data['is_weekend'] = False
new_date_data['is_holiday'] = False
new_date_data['is_close_from_weekend_or_holiday'] = False

# For every day in our time range
for current_date in new_date_data.index:
    if current_date.dayofweek >= 5:
        new_date_data.loc[current_date, 'is_weekend'] = True
    if current_date in bank_holidays:
        new_date_data.loc[current_date, 'is_holiday'] = True
    # Bridge between weekend and bank holiday
    # Rule: today is not weekend and yesterday and tomorrow are bank holidays or weekend
    yesterday = current_date - pd.to_timedelta(1, 'D')
    tomorrow = current_date + pd.to_timedelta(1, 'D')
    if (current_date.dayofweek < 5) and (
        yesterday.dayofweek >= 5 or yesterday in bank_holidays or tomorrow.dayofweek >= 5 or tomorrow in bank_holidays
    ):
        new_date_data.loc[current_date, 'is_close_from_weekend_or_holiday'] = True
        
new_date_data=new_date_data.reset_index()
new_date_data.rename(columns={'index': 'DateOfDeparture'}, inplace=True)

In [3]:
# Airport dataset containing coordinates and elevation
df = pd.read_csv(url+"airports.csv")

new_airport_data = pd.DataFrame(index=['ORD', 'LAS', 'DEN', 'ATL', 'SFO', 'EWR', 'IAH', 'LAX', 'DFW',
       'SEA', 'JFK', 'PHL', 'MIA', 'DTW', 'BOS', 'MSP', 'CLT', 'MCO',
       'PHX', 'LGA'], columns=['lat', 'lon', 'elevation'])
# Population in 2018 per cities according to
# https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population
new_airport_data['city_population'] = [
    2705994, 644644, 716492, 498044, 883305, 8398748, 2325502, 3990456, 1345047, 744955, 8398748, 1584138, 470914, 
    672662, 694583, 425403, 872498, 285713, 1660272, 8398748
]

for current_airport in new_airport_data.index:
    found = df.loc[
        np.logical_and(df['iso_country'] == 'US', df['local_code'] == current_airport),
        ['latitude_deg', 'longitude_deg', 'elevation_ft']
    ]
    new_airport_data.loc[current_airport, ['lat', 'lon', 'elevation']] = found.iloc[0,:].values


new_airport_data=new_airport_data.reset_index()
new_airport_data.rename(columns={'index': 'AirPort'}, inplace=True)

In [4]:
# Export to the external_data.csv
external_data = pd.read_csv(url+"old_external_data.csv")
external_data['Date']=pd.to_datetime(external_data['Date'])

In [5]:
USA = pd.read_excel(url+"USA.xlsx")

In [6]:
tab1= external_data.merge(new_date_data,how='left',left_on="Date",right_on="DateOfDeparture")
tab2 = USA.merge(new_airport_data,how='left',left_on="AirPort",right_on="AirPort")
final=tab1.merge(tab2,how="left",left_on="AirPort",right_on="AirPort")
final["Date"]=pd.to_datetime(final["Date"])
final.to_csv(url+"Ajout_données.csv", index=False)